In [1]:
import pandas as pd
import numpy as np
import plotly.offline as plt
import plotly.graph_objs as go
from boruta import BorutaPy
import optuna

import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split


# train = pd.read_csv('Final_train_data_with_363_features.csv')
# -------train = pd.read_csv('file:///D:/remote%20projects/Dr.%20Kenji%20Mizugichi/oversampling%20after%20feature%20selection/train_oversample_after_feature_selection.csv')
# train = pd.read_csv('file:///D:/remote%20projects/Dr.%20Kenji%20Mizugichi/oversampling%20after%20feature%20selection/train_oversample_before_feature_selection.csv')


# 0.01787708494254063 0.03594300058548619 100.01365326405414 23.036653198054267


train = pd.read_csv('file:///D:/remote%20projects/NIBIOHN/NSCLC/training%20files/412_genes/train_after_features_selection.csv')
train = train.drop(['Unnamed: 0'],axis=1)


try:
    train['SEPT4']

except:
    train.rename(columns={'Sep-04': 'SEPT4'}, inplace=True)
else:
    print('SEPT4 was present already')
    
  


from scipy import stats
import numpy as np
z = np.abs(stats.zscore(train))
threshold = 3
original_train = train
train = train[(z < 3).all(axis=1)]
train.columns
y = train['Disease Status (NSCLC: primary tumors; Normal: non-tumor lung tissues)']
x = train[train.columns[:-1]]
x = np.asarray(x)
y = np.asarray(y)
# from imblearn.over_sampling import ADASYN 
# sm = ADASYN(random_state = 2) 
# x, y = sm.fit_sample(x, y.ravel())


#clf = RandomForestRegressor(params)


   

from sklearn import linear_model
#clf = RandomForestRegressor(params)

#0.0303557625626358, 0.01, 10.0, 55.92073670515847
#

clf = linear_model.LogisticRegression(C=0.01787708494254063,penalty='l1', solver='liblinear',
                                      tol=0.03594300058548619, max_iter=100.01365326405414,
                                      warm_start=True,
                                      intercept_scaling=23.036653198054267,random_state = 2)
#from pprint import pprint
# Look at parameters used by our current forest
#print('Parameters currently in use:\n')
#pprint(rf.get_params())
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=3, random_state=42)
# X is the feature set and y is the target
fold = 1
for train_index, val_index in skf.split(x,y): 
    #print("Train:", train_index, "Validation:", val_index) 
    X_train, X_test = x[train_index], x[val_index] 
    Y_train, Y_test = y[train_index], y[val_index]
    model = clf.fit(X_train,Y_train)
    a_train = model.predict(X_train)
    a_test = model.predict(X_test)
    print('K-FOLD #',fold)
    fold+=1
    print('Train AUC-ROC Score: ',roc_auc_score(a_train,Y_train),' Val AUC-ROC score: ',roc_auc_score(a_test,Y_test),'\n')
    from sklearn.metrics import cohen_kappa_score

    print('validation kappa:-',cohen_kappa_score(a_test, Y_test))
    
   
# model = clf.fit(x[:800],y[:800])

# a_test = model.predict(x[800:])

# model = clf.fit(x,y)


from sklearn.metrics import classification_report, confusion_matrix
# print(confusion_matrix(a_test,y[800:]))
# print(classification_report(a_test,y[800:]))

print('\n\n Testing on the independent data-set\n\n')

# test = pd.read_csv('Final_independent_test_data_with_363_features.csv')
# ---test = pd.read_csv('file:///D:/remote%20projects/Dr.%20Kenji%20Mizugichi/oversampling%20after%20feature%20selection/test_oversample_after_feature_selection.csv')
# test = pd.read_csv('file:///D:/remote%20projects/Dr.%20Kenji%20Mizugichi/oversampling%20after%20feature%20selection/test_oversample_before_feature_selection.csv')

test = pd.read_csv('file:///D:/remote%20projects/NIBIOHN/NSCLC/test%20files/test_after_features_selection.csv')
test = test.drop(['Unnamed: 0'],axis=1)

try:
    test['SEPT4']

except:
    test.rename(columns={'Sep-04': 'SEPT4'}, inplace=True)
else:
    print('SEPT4 was present already')
    
  





y_testt = test['Disease Status (NSCLC: primary tumors; Normal: non-tumor lung tissues)']

x_testt = test[test.columns[:-1]]

y_testt_pred = model.predict(x_testt)

# prediction = []
# sum=0
# for i, j in y_testt_pred:
#     if i>j:
#         prediction.append(0)
#     else:
#         prediction.append(1)
#         sum+=1

# sum

a_test = []
for i in y_testt_pred:
    if i>0.5:
        a_test.append(1)
    else:
        a_test.append(0)
from sklearn.metrics import classification_report, confusion_matrix
print('0:- NSCLC Detected', '\t& \t1:- Normal\n')
print(confusion_matrix(y_testt,a_test))
print(classification_report(y_testt,a_test))

from sklearn.metrics import cohen_kappa_score

print('test kappa:-',cohen_kappa_score(y_testt, a_test))


SEPT4 was present already


C:\Users\SARANSH\Anaconda3\envs\saransh\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning:

Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.



K-FOLD # 1
Train AUC-ROC Score:  0.9350300459526335  Val AUC-ROC score:  0.9506002824858758 

validation kappa:- 0.7561177934467026
K-FOLD # 2
Train AUC-ROC Score:  0.9466337109234279  Val AUC-ROC score:  0.9195622860699098 

validation kappa:- 0.5357261108535858
K-FOLD # 3
Train AUC-ROC Score:  0.9400548785757497  Val AUC-ROC score:  0.941747572815534 

validation kappa:- 0.5940652818991099


 Testing on the independent data-set


SEPT4 was present already
0:- NSCLC Detected 	& 	1:- Normal

[[188  11]
 [  0  19]]
              precision    recall  f1-score   support

           0       1.00      0.94      0.97       199
           1       0.63      1.00      0.78        19

    accuracy                           0.95       218
   macro avg       0.82      0.97      0.87       218
weighted avg       0.97      0.95      0.95       218

test kappa:- 0.7486900020959966


In [2]:
from __future__ import division
import random
import math

#--- COST FUNCTION 
# function we are attempting to optimize (minimize)
def func1(x0,x,y,x_testt,y_testt):
    from sklearn import linear_model
    #clf = RandomForestRegressor(params)

    clf = linear_model.LogisticRegression(C=x0[0],penalty='l1', solver='liblinear',
                                          tol=x0[1], max_iter=x0[2],
                                          warm_start=True,
                                          intercept_scaling=x0[3],random_state = 2)
    #from pprint import pprint
    # Look at parameters used by our current forest
    #print('Parameters currently in use:\n')
    #pprint(rf.get_params())
#     print(clf)
    from sklearn.metrics import roc_auc_score
    from sklearn.model_selection import StratifiedKFold
#     model = clf.fit(x,y)
#     y_pred = model.predict(x_testt)
    print(x0[0], x0[1],x0[2], x0[3])
#     print('validation kappa:-',cohen_kappa_score(y_pred, y_testt))
    
#     a_train = model.predict(x)
#     a_test = model.predict(X_test)
    
    
    skf = StratifiedKFold(n_splits=3, random_state=42)
    # X is the feature set and y is the target
    fold = 1
    for train_index, val_index in skf.split(x,y): 
        #print("Train:", train_index, "Validation:", val_index) 
        X_train, X_test = x[train_index], x[val_index] 
        Y_train, Y_test = y[train_index], y[val_index]
        model = clf.fit(X_train,Y_train)
        a_train = model.predict(X_train)
        a_test = model.predict(X_test)
#         print('K-FOLD #',fold)
        fold+=1
#         print('Train AUC-ROC Score: ',roc_auc_score(a_train,Y_train),' Val AUC-ROC score: ',roc_auc_score(a_test,Y_test),'\n')
        from sklearn.metrics import cohen_kappa_score

#         print('validation kappa:-',cohen_kappa_score(a_test, Y_test))
        y_pred = model.predict(x_testt)
        print('Test kappa:-',cohen_kappa_score(y_pred, y_testt))
    
    

    return 1 - cohen_kappa_score(y_pred, y_testt)

#<-------------------------------------->



#--- MAIN 
class Particle:
    def __init__(self,x0):
        self.position_i=[]          # particle position
        self.velocity_i=[]          # particle velocity
        self.pos_best_i=[]          # best position individual
        self.err_best_i=-1          # best error individual
        self.err_i=-1               # error individual

        for i in range(0,num_dimensions):
            self.velocity_i.append(random.uniform(-1,1))
            self.position_i.append(x0[i])

    # evaluate current fitness
    def evaluate(self,costFunc,x,y,x_testt,y_testt):
        self.err_i=costFunc(self.position_i,x,y,x_testt,y_testt)

        # check to see if the current position is an individual best
        if self.err_i < self.err_best_i or self.err_best_i==-1:
            self.pos_best_i=self.position_i
            self.err_best_i=self.err_i

    # update new particle velocity
    def update_velocity(self,pos_best_g):
        w=0.1       # constant inertia weight (how much to weigh the previous velocity)
        c1=5       # cognative constant
        c2=10        # social constant

        for i in range(0,num_dimensions):
            r1=random.random()
            r2=random.random()

            vel_cognitive=c1*r1*(self.pos_best_i[i]-self.position_i[i])
            vel_social=c2*r2*(pos_best_g[i]-self.position_i[i])
            self.velocity_i[i]=w*self.velocity_i[i]+vel_cognitive+vel_social

    # update the particle position based off new velocity updates
    def update_position(self,bounds):
        for i in range(0,num_dimensions):
            self.position_i[i]=self.position_i[i]+self.velocity_i[i]

            # adjust maximum position if necessary
            if self.position_i[i]>bounds[i][1]:
                self.position_i[i]=bounds[i][1]

            # adjust minimum position if neseccary
            if self.position_i[i] < bounds[i][0]:
                self.position_i[i]=bounds[i][0]
                
class PSO():
    def __init__(self,costFunc,x,y,x_testt,y_testt,x0,bounds,num_particles,maxiter):
        global num_dimensions
        
        num_dimensions=len(x0)
        
        err_best_g=-1                   # best error for group
        pos_best_g=[]                   # best position for group

        # establish the swarm
        swarm=[]
        for i in range(0,num_particles):
            swarm.append(Particle(x0))
            

        # begin optimization loop
        i=0
        while i < maxiter:
            #print i,err_best_g
            # cycle through particles in swarm and evaluate fitness
            for j in range(0,num_particles):
#                 print('j')
                swarm[j].evaluate(costFunc,x,y,x_testt,y_testt)
                

                # determine if current particle is the best (globally)
                if swarm[j].err_i < err_best_g or err_best_g == -1:
                    pos_best_g=list(swarm[j].position_i)
                    err_best_g=float(swarm[j].err_i)

            # cycle through swarm and update velocities and position
            for j in range(0,num_particles):
                swarm[j].update_velocity(pos_best_g)
                swarm[j].update_position(bounds)
            i+=1

        # print final results
        print ('FINAL:')
        print (pos_best_g)
        print (err_best_g)

if __name__ == "__PSO__":
    main()
    

initial=[0.1,0.1,100,23]               # initial starting location [x1,x2...]
bounds=[(0.01,1),(0.01,1),(10,1000),(10,100)]  # input bounds [(x1_min,x1_max),(x2_min,x2_max)...]
PSO(func1,x,y,x_testt,y_testt,initial,bounds,num_particles=3,maxiter=30)

0.1 0.1 100 23


C:\Users\SARANSH\Anaconda3\envs\saransh\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning:

Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.



Test kappa:- 0.537003757555955
Test kappa:- 0.6511014874788175
Test kappa:- 0.5596700892105706
0.1 0.1 100 23


C:\Users\SARANSH\Anaconda3\envs\saransh\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning:

Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.



Test kappa:- 0.537003757555955
Test kappa:- 0.6511014874788175
Test kappa:- 0.5596700892105706
0.1 0.1 100 23


...

Test kappa:- 0.4495743759919203
Test kappa:- 0.848646146725295
Test kappa:- 0.6659643746408734
1.0 0.01 108.3933882685892 100


C:\Users\SARANSH\Anaconda3\envs\saransh\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning:

Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.



Test kappa:- 0.4495743759919203
Test kappa:- 0.848646146725295
Test kappa:- 0.6659643746408734
1.0 0.01 32.703673946692874 100


C:\Users\SARANSH\Anaconda3\envs\saransh\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning:

Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.



Test kappa:- 0.4495743759919203
Test kappa:- 0.848646146725295
Test kappa:- 0.6659643746408734
FINAL:
[1, 0.01, 100.64381620614017, 30.86308427154196]
0.33403562535912656


In [3]:
clf

LogisticRegression(C=0.01787708494254063, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=23.036653198054267,
                   l1_ratio=None, max_iter=100.01365326405414,
                   multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=2, solver='liblinear', tol=0.03594300058548619,
                   verbose=0, warm_start=True)